In [1]:
import time
import numpy as np
import os
import shutil

In [2]:
arr_dir = "npy"
shard_dir = "shard"

In [3]:
if os.path.exists(arr_dir):
    print("Deleting exitsting array directory. ")
    shutil.rmtree(arr_dir)
print("(re)creating new array directory")
os.makedirs(arr_dir)

if os.path.exists(shard_dir):
    print("Deleting exitsting shard directory. ")
    shutil.rmtree(shard_dir)
print("(re)creating new shard directory")
os.makedirs(shard_dir)

Deleting exitsting array directory. 
(re)creating new array directory
Deleting exitsting shard directory. 
(re)creating new shard directory


In [27]:
num_samples = 100 # number of sample files
X_shape = (640,640)

recipe_fn = "npy_recipe"
npy_recipe = open(recipe_fn, "w")

for i in range(num_samples):
    # create input array
    X = np.ones(X_shape) * i
    # create outpu array
    y = np.array(i)
    
    # set basename for npy input/output files
    arr_fn = "arr_%06d" % i
    # set npy filenames
    X_name = f"{arr_fn}.input.npy"
    y_name = f"{arr_fn}.output.npy"
    # save array
    np.save(os.path.join(arr_dir, X_name), X)
    np.save(os.path.join(arr_dir, y_name), y)
    
    # write sample information onto recipe file
    npy_recipe.write(f"{X_name}\tfile:{os.path.join(arr_dir, X_name)}\n")
    npy_recipe.write(f"{y_name}\tfile:{os.path.join(arr_dir, y_name)}\n")

npy_recipe.close()

 

生成されたinput/outputのnpyファイルをlsコマンドで確認します

In [28]:
!ls {arr_dir}

arr_000000.input.npy  arr_000033.output.npy arr_000067.input.npy
arr_000000.output.npy arr_000034.input.npy  arr_000067.output.npy
arr_000001.input.npy  arr_000034.output.npy arr_000068.input.npy
arr_000001.output.npy arr_000035.input.npy  arr_000068.output.npy
arr_000002.input.npy  arr_000035.output.npy arr_000069.input.npy
arr_000002.output.npy arr_000036.input.npy  arr_000069.output.npy
arr_000003.input.npy  arr_000036.output.npy arr_000070.input.npy
arr_000003.output.npy arr_000037.input.npy  arr_000070.output.npy
arr_000004.input.npy  arr_000037.output.npy arr_000071.input.npy
arr_000004.output.npy arr_000038.input.npy  arr_000071.output.npy
arr_000005.input.npy  arr_000038.output.npy arr_000072.input.npy
arr_000005.output.npy arr_000039.input.npy  arr_000072.output.npy
arr_000006.input.npy  arr_000039.output.npy arr_000073.input.npy
arr_000006.output.npy arr_000040.input.npy  arr_000073.output.npy
arr_000007.input.npy  arr_000040.output.npy arr_000074.input.npy
arr_000007.output.

レシピファイルの中身を確認します。ここでは`head`コマンドでレシピファイルの先頭部分を表示します

In [29]:
!head {recipe_fn}

arr_000000.input.npy	file:npy/arr_000000.input.npy
arr_000000.output.npy	file:npy/arr_000000.output.npy
arr_000001.input.npy	file:npy/arr_000001.input.npy
arr_000001.output.npy	file:npy/arr_000001.output.npy
arr_000002.input.npy	file:npy/arr_000002.input.npy
arr_000002.output.npy	file:npy/arr_000002.output.npy
arr_000003.input.npy	file:npy/arr_000003.input.npy
arr_000003.output.npy	file:npy/arr_000003.output.npy
arr_000004.input.npy	file:npy/arr_000004.input.npy
arr_000004.output.npy	file:npy/arr_000004.output.npy


この例では入力ファイルを`{basename}.input.npy`というファイルに、出力（正解）ファイルを`{basename}.output.npy`に出力しています。

その際、レシピファイルでは

{入力ファイル名} file:{入力ファイルまでのファイルパス} <BR>
{出力ファイル名} file:{出力ファイルまでのファイルパス}　<BR>
　・<BR>
　・<BR>
　・<BR>

といった形で出力していきます。


### create tar files using tarp command

`tarp`コマンドを使うと、レシピファイルからwebdataset形式のtarファイルを簡単に作成することができます。 <BR>
https://github.com/webdataset/tarp

In [30]:
!tarp -v create {recipe_fn} -o npy_webdataset.tar

[info] version  false
[info] 0 arr_000000.input.npy <- file:npy/arr_000000.input.npy
[info] 1 arr_000000.output.npy <- file:npy/arr_000000.output.npy
[info] 2 arr_000001.input.npy <- file:npy/arr_000001.input.npy
[info] 3 arr_000001.output.npy <- file:npy/arr_000001.output.npy
[info] 4 arr_000002.input.npy <- file:npy/arr_000002.input.npy
[info] 5 arr_000002.output.npy <- file:npy/arr_000002.output.npy
[info] 6 arr_000003.input.npy <- file:npy/arr_000003.input.npy
[info] 7 arr_000003.output.npy <- file:npy/arr_000003.output.npy
[info] 8 arr_000004.input.npy <- file:npy/arr_000004.input.npy
[info] 9 arr_000004.output.npy <- file:npy/arr_000004.output.npy
[info] 10 arr_000005.input.npy <- file:npy/arr_000005.input.npy
[info] 11 arr_000005.output.npy <- file:npy/arr_000005.output.npy
[info] 12 arr_000006.input.npy <- file:npy/arr_000006.input.npy
[info] 13 arr_000006.output.npy <- file:npy/arr_000006.output.npy
[info] 14 arr_000007.input.npy <- file:npy/arr_000007.input.npy
[info] 15 arr_

[info] 143 arr_000071.output.npy <- file:npy/arr_000071.output.npy
[info] 144 arr_000072.input.npy <- file:npy/arr_000072.input.npy
[info] 145 arr_000072.output.npy <- file:npy/arr_000072.output.npy
[info] 146 arr_000073.input.npy <- file:npy/arr_000073.input.npy
[info] 147 arr_000073.output.npy <- file:npy/arr_000073.output.npy
[info] 148 arr_000074.input.npy <- file:npy/arr_000074.input.npy
[info] 149 arr_000074.output.npy <- file:npy/arr_000074.output.npy
[info] 150 arr_000075.input.npy <- file:npy/arr_000075.input.npy
[info] 151 arr_000075.output.npy <- file:npy/arr_000075.output.npy
[info] 152 arr_000076.input.npy <- file:npy/arr_000076.input.npy
[info] 153 arr_000076.output.npy <- file:npy/arr_000076.output.npy
[info] 154 arr_000077.input.npy <- file:npy/arr_000077.input.npy
[info] 155 arr_000077.output.npy <- file:npy/arr_000077.output.npy
[info] 156 arr_000078.input.npy <- file:npy/arr_000078.input.npy
[info] 157 arr_000078.output.npy <- file:npy/arr_000078.output.npy
[info] 15

tarファイルが生成されているか`ls`コマンドで確認します

In [31]:
ls *.tar

npy_webdataset.tar


### tarファイルの分割（シャーディング）

大量のファイルをtarファイルにまとめることでデータ転送のオーバーヘッドが改善することができます。
しかし、一つのtarファイルに圧縮してしまうとファイル全体を転送するまで学習を始めることができなくなります。ここでは、tarファイルをいくつかのかたまりに分割する（シャーディング）することを考えます。かたまりのサイズはデータ転送効率の観点から100MB-1GBの間で設定することが多いようです。

シャードされたtarファイルを用いることで、tarファイルをオーバーヘッドなく転送して、なおかつシーケンシャルアクセスにより高速に読み込みます。データセットのシャッフルについては、シャードをシャッフルして読み込むことと、シーケンシャルリードしたシャードをシャッフルすることによって、データセット全体としてのシャッフルを保証します。


tarpを使えば、レシピファイルからシャーディングされたtarファイルを簡単に作成することができます。その場合、`パイプ`を用いることで大元のtarファイルの作成→分割を一つのコマンドラインで記述できます

コマンド中の`-`は標準出力/入力を表します。結果を標準出力に吐き出しそれを次のコマンドの入力としています。tarp splitコマンドでは`-c`オプションでファイルの数ごとにtarファイルを分割します。したがってデータの容量に従って`-c`オプションの数値を適切に調整することになります。ここでは10個のサンプルごとにtarファイルを分割しています（これはトイデータセットなので、このような分割は適切ではありません。あくまで動作を検証するためです）

出力するシャードは`%04d`のような記述をおこなうことで連番のファイルとして出力することが可能です。

In [32]:
!tarp -v create {recipe_fn} -o - | tarp split - -c 10 -o '{shard_dir}/npy_webdataset-%04d.tar'

[info] version  false
[info] version  false
[progress] # source -
[info] 0 arr_000000.input.npy <- file:npy/arr_000000.input.npy
[info] 1 arr_000000.output.npy <- file:npy/arr_000000.output.npy
[info] 2 arr_000001.input.npy <- file:npy/arr_000001.input.npy
[info] 3 arr_000001.output.npy <- file:npy/arr_000001.output.npy
[info] 4 arr_000002.input.npy <- file:npy/arr_000002.input.npy
[progress] # shard shard/npy_webdataset-0000.tar
[info] 5 arr_000002.output.npy <- file:npy/arr_000002.output.npy
[info] 6 arr_000003.input.npy <- file:npy/arr_000003.input.npy
[info] 7 arr_000003.output.npy <- file:npy/arr_000003.output.npy
[info] 8 arr_000004.input.npy <- file:npy/arr_000004.input.npy
[info] 9 arr_000004.output.npy <- file:npy/arr_000004.output.npy
[info] 10 arr_000005.input.npy <- file:npy/arr_000005.input.npy
[info] 11 arr_000005.output.npy <- file:npy/arr_000005.output.npy
[info] 12 arr_000006.input.npy <- file:npy/arr_000006.input.npy
[info] 13 arr_000006.output.npy <- file:npy/arr_000

[info] 131 arr_000065.output.npy <- file:npy/arr_000065.output.npy
[info] 132 arr_000066.input.npy <- file:npy/arr_000066.input.npy
[info] 133 arr_000066.output.npy <- file:npy/arr_000066.output.npy
[info] 134 arr_000067.input.npy <- file:npy/arr_000067.input.npy
[info] 135 arr_000067.output.npy <- file:npy/arr_000067.output.npy
[info] 136 arr_000068.input.npy <- file:npy/arr_000068.input.npy
[info] 137 arr_000068.output.npy <- file:npy/arr_000068.output.npy
[info] 138 arr_000069.input.npy <- file:npy/arr_000069.input.npy
[info] 139 arr_000069.output.npy <- file:npy/arr_000069.output.npy
[info] 140 arr_000070.input.npy <- file:npy/arr_000070.input.npy
[info] 141 arr_000070.output.npy <- file:npy/arr_000070.output.npy
[info] 142 arr_000071.input.npy <- file:npy/arr_000071.input.npy
[info] 143 arr_000071.output.npy <- file:npy/arr_000071.output.npy
[info] 144 arr_000072.input.npy <- file:npy/arr_000072.input.npy
[progress] # shard shard/npy_webdataset-0007.tar
[info] 145 arr_000072.outpu

作成されたシャード（tarファイル）を確認します。連番のtarファイルが作成されていることを確認します。

In [33]:
ls {shard_dir}

npy_webdataset-0000.tar  npy_webdataset-0004.tar  npy_webdataset-0008.tar
npy_webdataset-0001.tar  npy_webdataset-0005.tar  npy_webdataset-0009.tar
npy_webdataset-0002.tar  npy_webdataset-0006.tar
npy_webdataset-0003.tar  npy_webdataset-0007.tar


### upload sharded tar files to S3

作成したWebDataset形式のtarファイルは、ローカルフォルダ上に置いて機械学習のデータセットとして用いることもできます。しかし、WebDatasetが効果を発揮するのは、ファイル転送がボトルネックとなるようなクラウドストレージ上にデータを保存しているときです。

ここでは、作成したtarファイルをS3上にアップロードして動作検証していきます。


In [34]:
bucket = "taturabe-dataset"
prefix = "dataset/webdataset"
!aws s3 sync --delete {shard_dir} s3://{bucket}/{prefix}/{shard_dir}/

upload: shard/npy_webdataset-0000.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0000.tar
upload: shard/npy_webdataset-0002.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0002.tar
upload: shard/npy_webdataset-0001.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0001.tar
upload: shard/npy_webdataset-0004.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0004.tar
upload: shard/npy_webdataset-0003.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0003.tar
upload: shard/npy_webdataset-0005.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0005.tar
upload: shard/npy_webdataset-0006.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0006.tar
upload: shard/npy_webdataset-0009.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0009.tar
upload: shard/npy_webdataset-0007.tar to s3://taturabe-dataset/dataset/webdataset/shard/npy_webdataset-0